# Flair Pipeline mit pandas

## Importe

In [4]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter

import pickle
import string

import pandas as pd

## Laden der Daten

Die Daten, mit denen wir arbeiten, ist ein Korpus der die Reden der Bundesregierung seit Anfang der 1980er-Jahre bis 2017 umfasst.

Die Daten sind dem Projekt [German Political Speeches Corpus and Visualization](https://politische-reden.eu/) entnommen.

Als zip-Datei können die Daten auch hier heruntergeladen werden: https://zenodo.org/record/3611246

Es handelt sich dabei um die Datei Bundesregierung.xml

In [20]:
# Link ist bis zum 12.3.2022 verfügbar
url = 'https://box.hu-berlin.de/f/a2f4accf1cb1477bb505/?dl=1'

In [21]:
df = pd.read_xml(url)

In [22]:
# change dtype to datetime
df.loc[:, 'datum'] = pd.to_datetime(df.loc[:, 'datum'])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2983 entries, 0 to 2982
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   person      2983 non-null   object        
 1   titel       2982 non-null   object        
 2   datum       2983 non-null   datetime64[ns]
 3   untertitel  2088 non-null   object        
 4   url         2983 non-null   object        
 5   anrede      1447 non-null   object        
 6   rohtext     2983 non-null   object        
 7   ort         690 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 186.6+ KB


### Funktion zum Erstellen des Sentence-Objekts 

In [24]:
def create_sentence_object(text):
    '''
    Creates a Flair Sentence-Object.
    INPUT: string
    RETURN: flair.data.Sentence
    '''      
        
    return Sentence(text)    

In [25]:
%%time

df.loc[:, 'sentence_object'] = df.loc[:, 'rohtext'].apply(lambda text: create_sentence_object(text))

Wall time: 1min 44s


In [26]:
# speichern der Datei
# Größe der Datei 636 MB

# df.to_pickle('../data/flair-reden-bundesregierung.p')

## Tokenisierung

In [27]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
def tokenize(sentence):
    '''
    Tokenizes text using Sentence-Object
    INPUT: Sentence-Object
    RETURN: list with tokens
    '''    
    
    return [ token for token in sentence if str(token) not in string.punctuation ]

In [29]:
%%time

df.loc[:, 'tokens'] = df.loc[:, 'sentence_object'].apply(lambda sentence: tokenize(sentence))

Wall time: 4.75 s


In [30]:
df.loc[:, 'ntokens'] = df.loc[:, 'tokens'].apply(lambda tokens: len(tokens))

In [31]:
df.loc[:, 'ntokens'].describe()

count     2983.000000
mean      2190.128059
std       1458.956105
min          5.000000
25%       1093.000000
50%       1803.000000
75%       2934.000000
max      12217.000000
Name: ntokens, dtype: float64

In [32]:
test_token = df.loc[0, 'tokens']

In [33]:
type(test_token[0])

flair.data.Token

## Lemmatisierung

Derzeit noch nicht in Flair implementiert.

## NER

NER und POS-Tagging sind im Vergleich zu SpaCy sehr, sehr langsam. Eine GPU würde dies zwar beschleunigen, wäre dennoch deutlich langsamer.

Dafür wäre allerdings Accuracy bei der NER mit Flair um mehr als 10 % höher.


### Speed Comparison

In [34]:
rohtext = df.loc[0,'rohtext']

In [35]:
sentence_rohtext = Sentence(rohtext)

In [38]:
tagger = SequenceTagger.load('de-ner')

2022-02-23 16:02:51,363 --------------------------------------------------------------------------------
2022-02-23 16:02:51,366 The model key 'de-ner' now maps to 'https://huggingface.co/flair/ner-german' on the HuggingFace ModelHub
2022-02-23 16:02:51,367  - The most current version of the model is automatically downloaded from there.
2022-02-23 16:02:51,367  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/de-ner/de-ner-conll03-v0.4.pt)
2022-02-23 16:02:51,368 --------------------------------------------------------------------------------
2022-02-23 16:02:51,897 loading file C:\Users\nitra\.flair\models\ner-german\a125be40445295f7e94d0afdb742cc9ac40ec4e93259dc30f35220ffad9bf1f6.f46c4c5cfa5e34baa838983373e30051cd1cf1e933499408a49e451e784b0a11


In [39]:
%%time
# Das NER-Tagging ist sehr langsam!
# etwa 45 s für 2241 tokens

tagger.predict(sentence_rohtext)

Wall time: 41.7 s


In [40]:
len(sentence_rohtext)

2241

In [41]:
df.loc[:, 'ntokens'].sum()

6533152

### Berechnungen

https://spacy.io/usage/facts-figures#benchmarks-speed

Durchschnitt der Textlänge: 2190 Tokens

Dauer für 2241 Tokens 45 s

bei 2983 Dokument macht das ungefähr 37 Stunden!!!

## andere Berechnung aus SpaCy im Vergleich:

### Flair	

pos(-fast) & ner(-fast)

mit CPU Words per second 323	

dann ergeben sich 5,5 Stunden Laufzeit


mit GPU Words per second 1.184

dann ergäbe sich eine Laufzeit von 1,5 Stunden

allerdings bezieht sich das hier auf das englische ner-fast-Model, hier ist das normale deutsche Modell genutzt

### Spacy

Spacy benötigt für die Erstellung des Doc-Objekts 11 min 27 s, was genau mit der Speed Comparison übereinstimmt.

https://towardsdatascience.com/why-we-switched-from-spacy-to-flair-to-anonymize-french-legal-cases-e7588566825f

In [42]:
%%time

# Das Filtern nach den einzelnen Entities ist dann wiederum recht schnell.

list_per = [ token.text for token in sentence_rohtext if str(token.get_tag('ner')).split(' ')[0][-3:] == 'PER' ]

Wall time: 3 ms


In [43]:
list_per

['Hildegard',
 'Knef',
 'Billy',
 'Wilder',
 'Audrey',
 'Hepburn',
 'Robert',
 'de',
 'Niro',
 'Ernst',
 'Greta',
 'Garbo',
 'Ninotschka',
 'Achternbusch',
 'Dieter',
 'Kosslick',
 'Krzysztof',
 'Kieslowski',
 'Tom',
 'Tykwer']

### vorbereiteter Code für NER

In [27]:
def extract_named_entities(sentence, tagger, entity):
    '''
    Extracts named entities from Sentence-Object.
    INPUT: Sentence-Object
    RETURN: List with entities    
    '''    
    tagger.predict(sentence)
        
    return [ token.text for token in sentence if str(token.get_tag('de-ner')).split(' ')[0][-3:] == entity ]

In [1]:
%%time

entities = ['PER', 'ORG', 'LOC']
tagger = SequenceTagger.load('de-ner')

for entity in entities:
    df.loc[:, entity] = df.loc[:, 'rohtext'].apply(lambda text: extract_named_entities(text, tagger, entity))